# Building a Spam Filter with Naive Bayes
---
We're going to study the practical side of the algorithm by building a spam filter for SMS messages.

To classify messages as spam or non-spam, we saw in the previous mission that the computer:

-  Learns how humans classify messages.
- Uses that human knowledge to estimate probabilities for new messages — probabilities for spam and non-spam.
- Classifies a new message based on these probability values — if the probability for spam is greater, then it classifies the message as spam. Otherwise, it classifies it as non-spam (if the two probability values are equal, then we may need a human to classify the message).

So our first task is to "teach" the computer how to classify messages. To do that, we'll use the multinomial Naive Bayes algorithm along with a dataset of 5,572 SMS messages that are already classified by humans.

The dataset was put together by Tiago A. Almeida and José María Gómez Hidalgo, and it can be downloaded from the The UCI Machine Learning Repository. You can also download the dataset directly from [this link](https://dq-content.s3.amazonaws.com/433/SMSSpamCollection). The data collection process is described in more details on [this page](http://www.dt.fee.unicamp.br/~tiago/smsspamcollection/#composition), where you can also find some of the authors' papers.

### Importing the modules and dataset

In [1]:
#Importing the module
import pandas as pd

#Importing the dataset
spam=pd.read_csv('SMSSpamCollection',sep='\t',header=None,names=['Label','SMS'])

In [2]:
#A little peek at the dataset
print(spam.shape)
spam.sample(3,random_state=1)

(5572, 2)


,Label,SMS
1078,ham,"Yep, by the pretty sculpture"
4028,ham,"Yes, princess. Are you going to make me moan?"
958,ham,Welp apparently he retired


In [3]:
spam['Label'].value_counts(normalize=True)*100

ham     86.593683
spam    13.406317
Name: Label, dtype: float64

Around 87 percent of the messages are not spam (ham).

### Separating Training and Test Set

We're now going to split our dataset into a training and a test set, where the training set accounts for 80% of the data, and the test set for the remaining 20%.

In [4]:
randomized=spam.sample(frac=1,random_state=1)
train=randomized.iloc[:round(len(randomized)*0.8),:].reset_index(drop=True)
test=randomized.iloc[round(len(randomized)*0.8):,:].reset_index(drop=True)

In [5]:
#Checking the shape of training set
print(train.shape,'\n')

#Finding the percentage of ham in training set
train['Label'].value_counts(normalize=True)*100

(4458, 2) 



ham     86.54105
spam    13.45895
Name: Label, dtype: float64

In [6]:
#Checking the shape of test set
print(test.shape,'\n')

#Finding the percentage of ham in test set
test['Label'].value_counts(normalize=True)*100

(1114, 2) 



ham     86.804309
spam    13.195691
Name: Label, dtype: float64

The value of ham and spam is similar for both the training set and the test set compared to the original dataset (which is what we desired).
The results look good! We'll now move on to cleaning the dataset.

### Data Cleaning

To calculate all the probabilities required by the algorithm, we'll first need to perform a bit of data cleaning to bring the data in a format that will allow us to extract easily all the information we need.

Essentially, we want to bring data to this format:

![img](https://dq-content.s3.amazonaws.com/433/cpgp_dataset_3.png)


### Letter Case and Punctuation

We'll begin with removing all the punctuation and bringing every letter to lower case.

In [7]:
#Preview of pre-cleaned data
train.head(3)

,Label,SMS
0,ham,"Yep, by the pretty sculpture"
1,ham,"Yes, princess. Are you going to make me moan?"
2,ham,Welp apparently he retired


In [8]:
#Data cleaning
train['SMS']=train['SMS'].str.replace('\W',' ').str.lower()

#post-cleaned
train.head(3)

,Label,SMS
0,ham,yep by the pretty sculpture
1,ham,yes princess are you going to make me moan
2,ham,welp apparently he retired


### Creating the Vocabulary

Let's now move to creating the vocabulary, which in this context means a list with all the unique words in our training set.

In [9]:
train['SMS'] = train['SMS'].str.split()

vocabulary=[]

for i in train['SMS']:
    for j in i:
        vocabulary.append(j)
        
vocabulary=list(set(vocabulary))

print('There are {} unique words'.format(len(vocabulary)))

There are 7783 unique words


### Final Transformation

This is the final step to prepare the data. It will transform the data into our desired form above.

In [10]:
word_counts_per_sms = {unique_word: [0] * len(train['SMS']) for unique_word in vocabulary}

for index, sms in enumerate(train['SMS']):
    for word in sms:
        word_counts_per_sms[word][index] += 1

#Creating a dataframe
word=pd.DataFrame(word_counts_per_sms)

#Creating a new dataframe from training set and word
trainclean=pd.concat([train,word],axis=1)
trainclean.head(3)

,Label,SMS,0,00,000,000pes,008704050406,0089,01223585334,02,...,zindgi,zoe,zogtorius,zouk,zyada,é,ú1,ü,〨ud,鈥
0,ham,"[yep, by, the, pretty, sculpture]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ham,"[yes, princess, are, you, going, to, make, me,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ham,"[welp, apparently, he, retired]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Calculating Constants First

We're now done with cleaning the training set, and we can begin creating the spam filter. The Naive Bayes algorithm will need to answer these two probability questions to be able to classify new messages:

\begin{equation}
P(Spam | w_1,w_2, ..., w_n) \propto P(Spam) \cdot \prod_{i=1}^{n}P(w_i|Spam)
\end{equation}

\begin{equation}
P(Ham | w_1,w_2, ..., w_n) \propto P(Ham) \cdot \prod_{i=1}^{n}P(w_i|Ham)
\end{equation}


Also, to calculate P(w<sub>i</sub>|Spam) and P(w<sub>i</sub>|Ham) inside the formulas above, we'll need to use these equations:

\begin{equation}
P(w_i|Spam) = \frac{N_{w_i|Spam} + \alpha}{N_{Spam} + \alpha \cdot N_{Vocabulary}}
\end{equation}

\begin{equation}
P(w_i|Ham) = \frac{N_{w_i|Ham} + \alpha}{N_{Ham} + \alpha \cdot N_{Vocabulary}}
\end{equation}


Some of the terms in the four equations above will have the same value for every new message. We can calculate the value of these terms once and avoid doing the computations again when a new messages comes in. Below, we'll use our training set to calculate:

- P(Spam) and P(Ham)
- N<sub>Spam</sub>, N<sub>Ham</sub>, N<sub>Vocabulary</sub>

We'll also use Laplace smoothing and set $\alpha = 1$.

In [11]:
#Finding the probability of spam and ham
p_spam=trainclean['Label'].value_counts(normalize=True).loc['spam']
p_ham=trainclean['Label'].value_counts(normalize=True).loc['ham']

# N_Spam
n_words_per_spam_message = trainclean.loc[trainclean['Label']=='spam','SMS'].apply(len)
n_spam = n_words_per_spam_message.sum()

# N_Ham
n_words_per_ham_message = trainclean.loc[trainclean['Label']=='ham','SMS'].apply(len)
n_ham = n_words_per_ham_message.sum()

# N_Vocabulary
n_vocabulary = len(vocabulary)

#Alpha
alpha=1

### Calculating Parameters

Now that we have the constant terms calculated above, we can move on with calculating the parameters $P(w_i|Spam)$ and $P(w_i|Ham)$. Each parameter will thus be a conditional probability value associated with each word in the vocabulary.

The parameters are calculated using the formulas:

\begin{equation}
P(w_i|Spam) = \frac{N_{w_i|Spam} + \alpha}{N_{Spam} + \alpha \cdot N_{Vocabulary}}
\end{equation}

\begin{equation}
P(w_i|Ham) = \frac{N_{w_i|Ham} + \alpha}{N_{Ham} + \alpha \cdot N_{Vocabulary}}
\end{equation}

In [12]:
#Creating a dictionary for unique words
spam={i:0 for i in vocabulary}
ham={i:0 for i in vocabulary}

#Isolating the training set based on labels
t_spam=trainclean.loc[trainclean['Label']=='spam',:]
t_ham=trainclean.loc[trainclean['Label']=='ham',:]

#Updating the probabilities in the dictionary
for i in vocabulary:
    spam[i]=(t_spam[i].sum()+alpha)/(n_spam+alpha*n_vocabulary)
    ham[i]=(t_ham[i].sum()+alpha)/(n_ham+alpha*n_vocabulary)

### Classifying A New Message

Now that we have all our parameters calculated, we can start creating the spam filter. The spam filter can be understood as a function that:

- Takes in as input a new message (w<sub>1</sub>, w<sub>2</sub>, ..., w<sub>n</sub>).
- Calculates P(Spam|w<sub>1</sub>, w<sub>2</sub>, ..., w<sub>n</sub>) and P(Ham|w<sub>1</sub>, w<sub>2</sub>, ..., w<sub>n</sub>).
- Compares the values of P(Spam|w<sub>1</sub>, w<sub>2</sub>, ..., w<sub>n</sub>) and P(Ham|w<sub>1</sub>, w<sub>2</sub>, ..., w<sub>n</sub>), and:
    - If P(Ham|w<sub>1</sub>, w<sub>2</sub>, ..., w<sub>n</sub>) > P(Spam|w<sub>1</sub>, w<sub>2</sub>, ..., w<sub>n</sub>), then the message is classified as ham.
    - If P(Ham|w<sub>1</sub>, w<sub>2</sub>, ..., w<sub>n</sub>) < P(Spam|w<sub>1</sub>, w<sub>2</sub>, ..., w<sub>n</sub>), then the message is classified as spam.
    -  If P(Ham|w<sub>1</sub>, w<sub>2</sub>, ..., w<sub>n</sub>) = P(Spam|w<sub>1</sub>, w<sub>2</sub>, ..., w<sub>n</sub>), then the algorithm may request human help.

In [13]:
import re

def classify(message):

    message = re.sub('\W', ' ', message)
    message = message.lower()
    message = message.split()

    p_spam_given_message = p_spam
    p_ham_given_message = p_ham
    
    for i in message:
        if i in spam:
            p_spam_given_message*=spam[i]
        if i in ham:
            p_ham_given_message*=ham[i]
        
    print('P(Spam|message):', p_spam_given_message)
    print('P(Ham|message):', p_ham_given_message)

    if p_ham_given_message > p_spam_given_message:
        print('Label: Ham')
    elif p_ham_given_message < p_spam_given_message:
        print('Label: Spam')
    else:
        print('Equal proabilities, have a human classify this!')

In [14]:
#Sample spam case
classify('WINNER!! This is the secret code to unlock the money: C3421.')

P(Spam|message): 1.3481290211300841e-25
P(Ham|message): 1.9368049028589875e-27
Label: Spam


In [15]:
#Sample ham case
classify("Sounds good, Tom, then see u there")

P(Spam|message): 2.4372375665888117e-25
P(Ham|message): 3.687530435009238e-21
Label: Ham


### Measuring the Spam Filter's Accuracy

The two results above look promising, but let's see how well the filter does on our test set, which has 1,114 messages. The test set is practically new to the computer, it has never ever seen it (well technically it has, but never analyze it word for word).

We'll start by writing a function that returns classification labels instead of printing them.

In [16]:
def classify_test_set(message):

    message = re.sub('\W', ' ', message)
    message = message.lower()
    message = message.split()

    p_spam_given_message = p_spam
    p_ham_given_message = p_ham

    for word in message:
        if word in spam:
            p_spam_given_message *= spam[word]

        if word in ham:
            p_ham_given_message *= ham[word]

    if p_ham_given_message > p_spam_given_message:
        return 'ham'
    elif p_spam_given_message > p_ham_given_message:
        return 'spam'
    else:
        return 'needs human classification'

Now that we have a function that returns labels instead of printing them, we can use it to create a new column in our test set.

In [17]:
test['predicted'] = test['SMS'].apply(classify_test_set)
test.head()

,Label,SMS,predicted
0,ham,Later i guess. I needa do mcat study too.,ham
1,ham,But i haf enuff space got like 4 mb...,ham
2,spam,Had your mobile 10 mths? Update to latest Oran...,spam
3,ham,All sounds good. Fingers . Makes it difficult ...,ham
4,ham,"All done, all handed in. Don't know if mega sh...",ham


Now, we'll write a function to measure the accuracy of our spam filter to find out how well our spam filter does.

In [18]:
correct=0
total=test.shape[0]

for i in test.iterrows():
    i=i[1]
    if i['Label']==i['predicted']:correct+=1
        
print('''Total corrects is {},
Total mistakes {},
Accuracy is {}%'''.format(correct,total-correct,round(100*correct/total,3)))

Total corrects is 1100,
Total mistakes 14,
Accuracy is 98.743%


### Conclusion
After only targeting 80% accuracy, the result shown above is very good indeed since it is able to predict with close to 99 percent accuracy. Overall the model has learnt the training data well and has performed well on the test set for the text messages in the dataset used.